# Lakers Sentiment Analysis & Player Mention Analysis

## Comprehensive Analysis of Reddit Data with NER for Lakers Players

This notebook provides a comprehensive analysis of Lakers-related Reddit data, including:
- Sentiment analysis of posts and comments
- Named Entity Recognition (NER) for Lakers players
- Player-specific mention analysis
- Correlation analysis between sentiment and player performance
- Visualization and insights

---


## 1. Setup and Imports


In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Database connectivity
import psycopg2
from sqlalchemy import create_engine, text
import sys
import os

# Add project root to path
sys.path.insert(0, os.path.dirname(os.path.abspath('.')))

# Project imports
from data_processors.database_manager import DatabaseManager
from data_processors.ner_processor import LakersNERProcessor
from utils.lakers_roster import LAKERS_ROSTER, get_all_player_aliases
from utils.constants import DATABASE_HOST, DATABASE_NAME, DATABASE_PORT, DATABASE_USER, DATABASE_PASSWORD

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

# Statistical analysis
from scipy import stats
from scipy.stats import pearsonr, spearmanr
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Text analysis
import re
from collections import Counter, defaultdict
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer

# Download required NLTK data
try:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
    nltk.download('vader_lexicon', quiet=True)
except:
    pass

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All imports successful!")
print(f"📊 Available Lakers players: {len(LAKERS_ROSTER)}")
print(f"🔍 Player aliases loaded: {len(get_all_player_aliases())}")


## 2. Database Connection and Data Loading


In [ ]:
# Initialize database manager and NER processor
db_manager = DatabaseManager()
ner_processor = LakersNERProcessor()

print("🔌 Database connection established")
print("🧠 NER processor initialized")


In [ ]:
# Load Reddit posts data
posts_query = """
SELECT 
    id, title, selftext, author, subreddit, score, num_comments, 
    upvote_ratio, created_utc, over_18, edited, spoiler, stickied
FROM reddit_posts 
ORDER BY created_utc DESC
"""

with db_manager.engine.connect() as conn:
    posts_df = pd.read_sql(posts_query, conn)

print(f"📝 Loaded {len(posts_df)} Reddit posts")
print(f"📅 Date range: {posts_df['created_utc'].min()} to {posts_df['created_utc'].max()}")
posts_df.head()


In [ ]:
# Load Reddit comments data
comments_query = """
SELECT 
    id, post_id, author, body, score, created_utc, 
    parent_id, is_submitter, edited
FROM reddit_comments 
ORDER BY created_utc DESC
"""

with db_manager.engine.connect() as conn:
    comments_df = pd.read_sql(comments_query, conn)

print(f"💬 Loaded {len(comments_df)} Reddit comments")
print(f"📅 Date range: {comments_df['created_utc'].min()} to {comments_df['created_utc'].max()}")
comments_df.head()


In [ ]:
# Load sentiment analysis data
sentiment_query = """
SELECT 
    post_id, player_name, vader_positive, vader_negative, vader_neutral, vader_compound,
    transformer_positive, transformer_negative, transformer_neutral, transformer_sentiment,
    analysis_timestamp
FROM sentiment_scores 
ORDER BY analysis_timestamp DESC
"""

with db_manager.engine.connect() as conn:
    sentiment_df = pd.read_sql(sentiment_query, conn)

print(f"😊 Loaded {len(sentiment_df)} sentiment analysis records")
print(f"👥 Unique players mentioned: {sentiment_df['player_name'].nunique()}")
sentiment_df.head()


## 3. Named Entity Recognition (NER) Analysis


In [ ]:
# Process posts for player mentions
print("🔍 Processing posts for player mentions...")
processed_posts, post_mentions = ner_processor.process_reddit_posts(posts_df)

print(f"✅ Found {len(post_mentions)} player mentions in posts")
print(f"📊 Mentions by player:")
if not post_mentions.empty:
    mention_summary = post_mentions['player_name'].value_counts()
    print(mention_summary.head(10))
else:
    print("No player mentions found in posts")


In [ ]:
# Process comments for player mentions
print("🔍 Processing comments for player mentions...")
processed_comments, comment_mentions = ner_processor.process_reddit_comments(comments_df)

print(f"✅ Found {len(comment_mentions)} player mentions in comments")
print(f"📊 Mentions by player:")
if not comment_mentions.empty:
    comment_mention_summary = comment_mentions['player_name'].value_counts()
    print(comment_mention_summary.head(10))
else:
    print("No player mentions found in comments")


In [ ]:
# Combine all mentions
all_mentions = pd.concat([post_mentions, comment_mentions], ignore_index=True)
print(f"📊 Total player mentions: {len(all_mentions)}")
print(f"👥 Unique players mentioned: {all_mentions['player_name'].nunique()}")

# Display top mentioned players
if not all_mentions.empty:
    top_players = all_mentions['player_name'].value_counts().head(15)
    print("\n🏆 Top 15 Most Mentioned Players:")
    for player, count in top_players.items():
        print(f"  {player}: {count} mentions")
else:
    print("No mentions found")


## 4. Player-Specific Analysis


In [ ]:
# Create player-specific analysis function
def analyze_player_mentions(player_name, mentions_df, posts_df, comments_df):
    """
    Comprehensive analysis for a specific player
    """
    player_mentions = mentions_df[mentions_df['player_name'] == player_name].copy()
    
    if player_mentions.empty:
        return {
            'player_name': player_name,
            'total_mentions': 0,
            'post_mentions': 0,
            'comment_mentions': 0,
            'avg_confidence': 0,
            'contexts': [],
            'sentiment_data': pd.DataFrame()
        }
    
    # Separate post and comment mentions
    post_mentions = player_mentions[player_mentions['post_id'].notna()]
    comment_mentions = player_mentions[player_mentions['comment_id'].notna()]
    
    # Get sentiment data for this player
    player_sentiment = sentiment_df[sentiment_df['player_name'] == player_name]
    
    return {
        'player_name': player_name,
        'total_mentions': len(player_mentions),
        'post_mentions': len(post_mentions),
        'comment_mentions': len(comment_mentions),
        'avg_confidence': player_mentions['confidence'].mean(),
        'contexts': player_mentions['context'].tolist(),
        'sentiment_data': player_sentiment
    }

# Analyze each Lakers player
player_analyses = {}
for player_name in LAKERS_ROSTER.keys():
    player_analyses[player_name] = analyze_player_mentions(
        player_name, all_mentions, posts_df, comments_df
    )

print("✅ Player analysis completed for all Lakers players")
print(f"📊 Players with mentions: {sum(1 for p in player_analyses.values() if p['total_mentions'] > 0)}")
print(f"📊 Players without mentions: {sum(1 for p in player_analyses.values() if p['total_mentions'] == 0)}")


In [ ]:
# Create comprehensive player summary
player_summary_data = []
for player_name, analysis in player_analyses.items():
    player_info = LAKERS_ROSTER[player_name]
    
    # Calculate sentiment metrics if available
    sentiment_metrics = {}
    if not analysis['sentiment_data'].empty:
        sentiment_data = analysis['sentiment_data']
        sentiment_metrics = {
            'avg_vader_compound': sentiment_data['vader_compound'].mean(),
            'avg_transformer_positive': sentiment_data['transformer_positive'].mean(),
            'avg_transformer_negative': sentiment_data['transformer_negative'].mean(),
            'positive_mentions': len(sentiment_data[sentiment_data['vader_compound'] > 0.05]),
            'negative_mentions': len(sentiment_data[sentiment_data['vader_compound'] < -0.05]),
            'neutral_mentions': len(sentiment_data[(sentiment_data['vader_compound'] >= -0.05) & 
                                                 (sentiment_data['vader_compound'] <= 0.05)])
        }
    
    player_summary_data.append({
        'player_name': player_name,
        'position': player_info['position'],
        'jersey_number': player_info['jersey_number'],
        'total_mentions': analysis['total_mentions'],
        'post_mentions': analysis['post_mentions'],
        'comment_mentions': analysis['comment_mentions'],
        'avg_confidence': analysis['avg_confidence'],
        **sentiment_metrics
    })

player_summary_df = pd.DataFrame(player_summary_data)
player_summary_df = player_summary_df.fillna(0)

print("📊 Lakers Player Mention Summary:")
print(player_summary_df.sort_values('total_mentions', ascending=False).to_string(index=False))


## 5. Visualization Dashboard


In [ ]:
# Create comprehensive visualization dashboard
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        'Player Mention Frequency',
        'Sentiment Distribution',
        'Player Sentiment Scores',
        'Mention Confidence Distribution',
        'Post vs Comment Mentions',
        'Sentiment Over Time'
    ),
    specs=[[{"type": "bar"}, {"type": "histogram"}],
           [{"type": "bar"}, {"type": "histogram"}],
           [{"type": "bar"}, {"type": "scatter"}]]
)

# 1. Player Mention Frequency
if not all_mentions.empty:
    mention_counts = all_mentions['player_name'].value_counts().head(10)
    fig.add_trace(
        go.Bar(x=mention_counts.index, y=mention_counts.values, name="Mentions"),
        row=1, col=1
    )

# 2. Sentiment Distribution
if not sentiment_df.empty:
    fig.add_trace(
        go.Histogram(x=sentiment_df['vader_compound'], name="Vader Compound"),
        row=1, col=2
    )

# 3. Player Sentiment Scores
if not sentiment_df.empty:
    player_sentiment = sentiment_df.groupby('player_name')['vader_compound'].mean().sort_values(ascending=False).head(10)
    fig.add_trace(
        go.Bar(x=player_sentiment.index, y=player_sentiment.values, name="Avg Sentiment"),
        row=2, col=1
    )

# 4. Mention Confidence Distribution
if not all_mentions.empty:
    fig.add_trace(
        go.Histogram(x=all_mentions['confidence'], name="Confidence"),
        row=2, col=2
    )

# 5. Post vs Comment Mentions
if not all_mentions.empty:
    post_mentions_count = len(all_mentions[all_mentions['post_id'].notna()])
    comment_mentions_count = len(all_mentions[all_mentions['comment_id'].notna()])
    
    fig.add_trace(
        go.Bar(x=['Posts', 'Comments'], y=[post_mentions_count, comment_mentions_count], name="Mention Type"),
        row=3, col=1
    )

# 6. Sentiment Over Time
if not sentiment_df.empty:
    sentiment_df['date'] = pd.to_datetime(sentiment_df['analysis_timestamp']).dt.date
    daily_sentiment = sentiment_df.groupby('date')['vader_compound'].mean().reset_index()
    
    fig.add_trace(
        go.Scatter(x=daily_sentiment['date'], y=daily_sentiment['vader_compound'], 
                  mode='lines+markers', name="Daily Sentiment"),
        row=3, col=2
    )

# Update layout
fig.update_layout(
    height=1200,
    title_text="Lakers Sentiment Analysis Dashboard",
    showlegend=False
)

fig.show()


## 6. Export Results and Summary
